# Anomaly detection in fiber optic polarization
In this example we demonstrate the capabilities of Learning Entropy on anomaly detection of real data measured by CzechLight Polarilog [1]

[1] R. Vohnout, M. Šlapák, J. Jedlinský, and J. Vojtěch, “CzechLight Polarilog - Rapid Polarization Rotation Monitoring Appliance,” in 2020 22nd International Conference on Transparent Optical Networks (ICTON), Jul. 2020, pp. 1–4. doi: 10.1109/ICTON51198.2020.9203142.
